# Summary
This document corresponds to Exercise 11 of [this file](https://github.com/PerformanceEstimation/Learning-Performance-Estimation/blob/main/Course.pdf).

If [PEPit](https://pypi.org/project/PEPit/) is not already installed, please execute the following cell.

In [ ]:
!pip install pepit

### Exercise 11.1

Compute the worst-case ratio $\frac{\|\mathrm{Proj}_{Q_1}(x_k)-\mathrm{Proj}_{Q_2}(x_k)\|^2}{\|x_0-x_\star\|^2}$ for the different methods:
(i) Alternate projections,
(ii) Averaged projections,
(iii) Dykstra,
(iv) Douglas-Rachford,
(v) Peaceman-Rachford.

In [ ]:
from PEPit import PEP
from PEPit import Point
from PEPit.functions import SmoothConvexFunction
from PEPit.functions import ConvexIndicatorFunction
from PEPit.primitive_steps import proximal_step
from PEPit.primitive_steps import linear_optimization_step

def alternate_projection(n, verbose):
    # Instantiate PEP
    problem = PEP()

    f1 = problem.declare_function(ConvexIndicatorFunction)
    f2 = problem.declare_function(ConvexIndicatorFunction)
    func = f1 + f2

    # Start by defining its unique optimal point xs = x_*
    xs = func.stationary_point()

    # Then define the starting point x0 of the algorithm
    x0 = problem.set_initial_point()

    # Run the alternate projection method
    x = x0
    y = x0
    for _ in range(n):
        y, _, _ = proximal_step(x, f1, 1)
        x, _, _ = proximal_step(y, f2, 1)

    # Set the performance metric
    proj1_x, _, _ = proximal_step(x, f1, 1)
    proj2_x = x
    problem.set_performance_metric((proj2_x-proj1_x)**2)
    problem.set_initial_condition((x0-xs)**2<=1)

    # Solve the PEP
    pepit_tau = problem.solve(verbose)
    
    return pepit_tau

def averaged_projection(n, verbose):
    # Instantiate PEP
    problem = PEP()

    f1 = problem.declare_function(ConvexIndicatorFunction)
    f2 = problem.declare_function(ConvexIndicatorFunction)
    func = f1 + f2

    # Start by defining its unique optimal point xs = x_*
    xs = func.stationary_point()

    # Then define the starting point x0 of the algorithm
    x0 = problem.set_initial_point()

    # Run the alternate projection method
    x = x0
    y = x0
    for _ in range(n):
        y1, _, _ = proximal_step(x, f1, 1)
        y2, _, _ = proximal_step(x, f2, 1)
        x = 1/2 * (y1 + y2)

    # Set the performance metric
    proj1_x, _, _ = proximal_step(x, f1, 1)
    proj2_x, _, _  = proximal_step(x, f2, 1)
    problem.set_performance_metric((proj2_x-proj1_x)**2)
    problem.set_initial_condition((x0-xs)**2<=1)

    # Solve the PEP
    pepit_tau = problem.solve(verbose)
    
    return pepit_tau

def Dykstra(n, verbose):
    # Instantiate PEP
    problem = PEP()

    f1 = problem.declare_function(ConvexIndicatorFunction)
    f2 = problem.declare_function(ConvexIndicatorFunction)
    func = f1 + f2

    # Start by defining its unique optimal point xs = x_*
    xs = func.stationary_point()

    # Then define the starting point x0 of the algorithm
    x0 = problem.set_initial_point()

    # Run the alternate projection method
    x = x0
    y = x0
    p = 0*x0 # initiate a null point (to be corrected)
    q = 0*x0 # initiate a null point (to be corrected)
    for _ in range(n):
        y, _, _ = proximal_step(x + p, f1, 1)
        p = x + p - y
        x, _, _ = proximal_step(y + q, f2, 1)
        q = y + q - x

    # Set the performance metric
    proj1_x, _, _ = proximal_step(x, f1, 1)
    proj2_x = x
    problem.set_performance_metric((proj2_x-proj1_x)**2)
    problem.set_initial_condition((x0-xs)**2<=1)

    # Solve the PEP
    pepit_tau = problem.solve(verbose)
    
    return pepit_tau

Test a few values

In [ ]:
n = 4
verbose = 0
res1 = alternate_projection(n,verbose)
res2 = averaged_projection(n,verbose)
res3 = Dykstra(n, verbose)


Compute a few values and plot them as a function of the iteration counter.

In [ ]:
import numpy as np
import time
import matplotlib.pyplot as plt

n_max = 10
n_list = list(range(1,n_max))
L = 1
verbose = 0

pepit_taus_AltProj = list()
pepit_taus_AvgProj = list()
pepit_taus_DykProj = list()

for i in range(1,n_max):
    t0= time.process_time()
    pepit_tau = alternate_projection(i,verbose)
    pepit_taus_AltProj.append(pepit_tau)
    pepit_tau = averaged_projection(i,verbose)
    pepit_taus_AvgProj.append(pepit_tau)
    pepit_tau = Dykstra(i,verbose)
    pepit_taus_DykProj.append(pepit_tau)
    t1 = time.process_time() - t0
    print(i, '/', n_max-1,' done (elapsed time:',"%.2f" %t1,'[s])')

In [ ]:
plt.plot(n_list, pepit_taus_AltProj, '-', label='Alt')
plt.plot(n_list, pepit_taus_AvgProj, '-', label='Avg')
plt.plot(n_list, pepit_taus_DykProj, '-', label='Dykstra')

plt.legend()
plt.xlabel('N')
plt.loglog()
plt.show()